In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 100)

In [3]:
# Load CSV files
dfg = pd.read_csv('group.csv', low_memory=False)
dfh = pd.read_csv('hostels.csv', low_memory=False)

allocation = []

# Iterate through each group
for _, group in dfg.iterrows():
    group_id = group['Group ID ']
    members = group['Members']
    gender = group['Gender']

    if '&' in gender:
        genders = gender.split('&')
        gender_count = [int(s.split()[0]) for s in genders]
        gender_type = [s.split()[1] for s in genders]
    else:
        gender_count = [members]
        gender_type = [gender]

    for count, g in zip(gender_count, gender_type):
        while count > 0:
            avail_rooms = dfh[(dfh['Gender'] == g) & (dfh['Capacity'] >= count)]
            avail_rooms = avail_rooms.sort_values(by='Capacity')

            if not avail_rooms.empty:
                room = avail_rooms.iloc[0]
                allocation.append({
                    'Group ID': group_id,
                    'Hostel Name': room['Hostel Name '],
                    'Room Number': room['Room Number '],
                    'Members Allocated': count,
                    'Members remaining': 0,
                    # 'Capacity': room['Capacity']
                })
                dfh.at[room.name, 'Capacity'] -= count
                if dfh.at[room.name, 'Capacity'] == 0:
                    dfh = dfh.drop(room.name)
                break
            else:
                avail_rooms = dfh[(dfh['Gender'] == g) & (dfh['Capacity'] <= count)]
                if not avail_rooms.empty:
                    avail_rooms = avail_rooms.sort_values(by='Capacity', ascending=False)
                    room = avail_rooms.iloc[0]
                    allocation.append({
                        'Group ID': group_id,
                        'Hostel Name': room['Hostel Name '],
                        'Room Number': room['Room Number '],
                        'Members Allocated': room['Capacity'],
                        'Members remaining': 0,
                        # 'Capacity': room['Capacity']
                    })
                    count -= room['Capacity']
                    dfh = dfh.drop(room.name)
                    # count = 0
                else:
                    allocation.append({
                        'Group ID': group_id,
                        'Hostel Name': 'NA',
                        'Room Number': 'NA',
                        'Members Allocated': 0,
                        'Members remaining': count,
                        # 'Capacity': 'NA'
                    })
                    count = 0

# Convert the allocation list to a DataFrame and save it as a CSV file
allocation_df = pd.DataFrame(allocation)
# allocation_df.to_csv('room_allocation.csv', index=False)

# Print allocation DataFrame for verification
print(allocation_df)


   Group ID Hostel Name Room Number  Members Allocated  Members remaining
0       101         BH1         101                  3                  0
1       102         GH2         202                  4                  0
2       103         BH2         102                  2                  0
3       104         GH1         201                  2                  0
4       104         GH2         202                  1                  0
5       104          NA          NA                  0                  2
6       105         BH2         102                  2                  0
7       105          NA          NA                  0                  3
8       105          NA          NA                  0                  3


In [22]:
allocation_df.to_csv('se.csv')